# 高级搜索

对问题进行求解，实际上是在问题对应的状态空间中进行映射与遍历

## 复习：蛮力搜索的3种基本类型

指数型：K**n<br>
排列型：n!<br>
组合型：n!/(m!*(n-m)!)

#### 92. 递归实现指数型枚举 https://www.acwing.com/problem/content/94/

In [ ]:
n=int(input())
res=[]
def dfs(idx,nums,start):
    if idx==n:
        res.append(nums)
        return
    for i in range(start,n+1):
        dfs(idx+1,nums,i+1)
        dfs(idx+1,nums+[i],i+1)
for i in range(1,n+1):
    dfs(1,[],i+1)
    dfs(1,[i],i+1)
for r in res:
    print(" ".join([str(i) for i in r]))

#### 94. 递归实现排列型枚举 https://www.acwing.com/problem/content/96/

In [ ]:
n=int(input())
res=[]
def dfs(idx,num):
    if idx==n:
        res.append(num)
        return
    for i in range(n):
       if not visit[i]:
           visit[i]=True
           dfs(idx+1,num+[i+1])
           visit[i]=False
visit=[False]*n
for i in range(n):
    visit[i]=True
    dfs(1,[i+1])
    visit[i]=False
for r in res:
    print(" ".join([str(t) for t in r]))

#### 93. 递归实现组合型枚举 https://www.acwing.com/problem/content/95/

In [ ]:
a=input()
n,m=[int(i) for i in a.split()]
def solution(n,m):
    if m==0:
        return
    res=[]
    def dfs(idx,nums):
        if idx==m:
            res.append(nums)
            return
        for i in range(nums[-1]+1,n+2-(m-idx)):
            dfs(idx+1,nums+[i])
    for i in range(1,n+2-m):
        dfs(1,[i])
    for r in res:
        print(" ".join([str(t) for t in r]))
solution(n,m)

## 复习：初级搜索

搜索方向：<br>
1、深度优先（与递归和栈相关）<br>
2、广度优先（与队列相关）<br>

简单的优化：<br>
判重（避免重复搜索某一状态）

状态空间、搜索树/图<br>
DFS/BFS对状态空间的遍历形成一棵树

#### 165. 小猫爬山 https://www.acwing.com/problem/content/167/

In [ ]:
n,w=map(int, input().split())
arr=[]
for i in range(n):
    arr.append(int(input()))
arr.sort(reverse=True)
ans=float("inf")
s=[0]*n
def dfs(idx,k):
    global ans
    if k>=ans:
        return
    if idx==n:
        ans=k
        return
    for i in range(k):
        if s[i]+arr[idx]<=w:
            s[i]+=arr[idx]
            dfs(idx+1,k)
            s[i]-=arr[idx]
    s[k]+=arr[idx]
    dfs(idx+1,k+1)
    s[k]-=arr[idx]
dfs(0,1)
print(ans)

## 搜索剪枝

蛮力搜索（回溯）作为最原始的遍历状态空间的方法，本质上是试错<br>
一个分支不可行，就需要取消上一步甚至前几步的计算，换个分支重新来过<br>
在分之较多、较深的问题中，很容易导致复杂度为指数时间的运算

剪枝，就是通过已有的信息，提前判定某些分支不可行或一定不优，从而减少需要访问的状态数量，形象地说就像剪去“搜索树”的枝条，所以叫剪枝

在深度优先搜索中，有以下常见剪枝方法：<br>
1、优化搜索顺序<br>
2、排除等效冗余（如果能够判定从搜索树的当前节点沿着某几条不同分支到达子树是等效的，那么只需对其中一条分支执行搜索）<br>
3、可行性剪枝（在搜索过程中，及时对当前状态进行检查，如果发现分支已经无法到达递归边界，就执行回溯）<br>
4、最优性剪枝（在最优化问题中吗，如果当前花费的代价已经超过了当前搜索的最优解，可以停止对当前分支的搜索，或者可以判定该分支比最优解差，也可以剪枝，滑动窗口、单调栈、贪心都是该方法的应用）<br>
5、记忆化

## 迭代加深、折半搜索、双向搜索

### 迭代加深

对DFS的优化<br>
适用于搜索树最大深度很大，但答案可能并不太深的问题<br>
可以防止DFS一开始选错了分支，在不包含答案的深层子树上浪费许多时间<br>

当搜索树规模随着层次的深入增长很快，并且我们能够确保答案在一个较浅层的节点时，就可以采用迭代加深的深度优先搜索算法来解决问题。<br>
“迭代加深”思想就是从小到大限制搜索的深度，如果当前深度限制下搜不到答案，就把深度限制增加，重新进行一次搜索

#### 170. 加成序列 https://www.acwing.com/problem/content/172/

In [ ]:
def dfs(u,depth):
    if u==depth:
        return path[u-1]==a
    visit=set()
    for i in range(u-1,-1,-1):
        for j in range(i,-1,-1):
            s=path[i]+path[j]
            if s>path[u-1] and s<=a and s not in visit:
                visit.add(s)
                if len(path)<=u:
                    path.append(s)
                else:
                    path[u]=s
                if dfs(u+1,depth):
                    return True
    return False

while True:
    a=int(input())
    if a==0:
        break
    path=[1]
    depth=1
    while not dfs(1,depth):
        depth+=1
    print(" ".join(str(i) for i in path))

### 双向BFS

适用于层数不太深，但每层分支数量大的问题<br>
从初态和终态出发各搜索一半状态，产生两棵深度减半的搜索树<br>
在中间交会，用适当方法合并成最终的答案

### 折半搜索

与双向BFS非常类似<br>
双向BFS:有确定的起点、终点，同时开始，每边搜一步<br>
折半搜索:用于集合类的问题，把集合分成两半，分别搜出所有方案，再合起来<br>

## 启发式搜索：A*算法

A*算法是一种启发式搜索(Heuristically Search）算法<br>
A*算法的关键是设计一个估价函数:<br>
·以任意“状态”为输入，计算出从该状态到目标状态所需代价的估计值<br>
·在搜索中，维护一个堆（优先队列)，优先选择“当前代价+未来估价”最小的状态进行扩展<br>
估价函数的设计原则:估值必须比实际更优（估计代价≤未来实际代价)<br>
只要保证上述原则，当目标状态第一次从堆中被取出时，就得到了最优解<br>

把好状态估差的后果:<br>
本来在最优解搜索路径上的状态被错误地估计了较大的代价，被压在堆中无法取出，从而导致非最优解搜索路径上的状态不断扩展，直至在目标状态上产生错误的答案
把坏状态估好的后果:<br>
只要估价不大于未来实际代价，这个值总会比最优解更早地被取出，从而得到修正。最坏后果无非就是算的状态多了,跑得慢一些。<br>
否决一个正确idea vs多看一个垃圾idea<br>

A*和优先队列BFS的区别就是:考虑优先级的时候有没有加上未来估价<br>
估价越精准（接近但不超过未来实际代价)，A*算法越快<br>
估价等于0，就退化为了优先队列BFS<br>
A*算法的关键:开动脑筋，设计优秀的估价函数（必须要乐观估计，但也要尽量精准)<br>

# 平衡二叉树

## 二叉搜索树Binary Search Tree

二叉搜索树（Binary Search Tree)是一棵满足如下性质（BST性质)的二叉树:<br>
·任意结点的关键码≥它左子树中所有结点的关键码<br>
·任意结点的关键码≤它右子树中所有结点的关键码<br>
根据以上性质，二叉搜索树的中序遍历必然为一个有序序列<br>

## AVL树


平衡因子Balance Factor :<br>
一个结点的左子树的高度减去它的右子树的高度。<br>


AVL树<br>
·任意结点的平衡因子的绝对值都不超过1，即balance factor E {-1,0,1}<br>
·每个结点需要保存:原始数据、左子结点、右子结点、子树高度<br>
AVL树在插入、删除时，沿途更新结点的高度值<br>
当平衡因子的绝对值大于1时，触发树结构的修正，通过旋转操作来进行平衡<br>

## 红黑树

红黑树(Red-black Tree）是一种近似平衡的二叉搜索树<br>
·从根到叶子的最长路径≤2*最短路径（简记:高度差在2倍以内)<br>

规则:<br>
·每个结点要么是红色，要么是黑色<br>
·根结点是黑色<br>
·最底层视作有一层空叶结点，是黑色的<br>
·不能有两个相邻的红色结点<br>
·从任一结点到其每个叶子的所有路径都包含相同数目的黑色结点<br>

# 跳表

跳表（Skip List）是对元素有序的链表的优化，对标的是平衡树和二分查找<br>
二分查找:可以在数组上O(logN)查询，不可修改序列（不能用于链表<br>
平衡树:支持高效的查询、插入、删除，但比较复杂，不容易实现<br>
跳表是一种查询、插入、删除都是O(logN)的数据结构，其特点是原理简单、容易实现、方便扩展、效率优秀，在 Redis、LevelDB等热门项目中用于代替平衡树。
链表插入、删除都是O(1)，但查询很慢——O(N)<br>
跳表的核心思想:如何提高有序链表的查询效率?<br>


# 树堆

Treap = Tree + Heap<br>
树堆(Treap）的每个结点保存两个值<br>
·原始数据，也叫关键码<br>
·额外的权值，是随机生成的<br>
树堆首先是一棵二叉搜索树，结点的关键码（原始数据）满足BST性质:左≤根≤右<br>
树堆也是一个堆，结点的额外权值满足大根堆形式:父≥子<br>
Treap各项操作的时间复杂度均为O(logN)<br>
Treap检索、求前驱、求后继的操作与普通BST一致——一次递归查找<br>